In [8]:
import requests
import pandas as pd
import json

In [9]:
api_key = "df6f62c7bff5489ea030bcd96b725280"
accident = pd.read_csv('Road Accident Data.csv')
sample_size = int(len(accident)* 0.0001)
accident = accident.sample(n=sample_size, random_state=42)


In [ ]:
url = "http://api.weatherstack.com/historical"

weather_data = []

for _, row in accident.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    date = row['Accident Date']
    index = row["Accident_Index"]
    

    # Paramètres pour l'API Weatherstack
    params = {
        "access_key": api_key,
        "query": f"{latitude},{longitude}",
        "historical_date": date
    }
    
    try:
        response = requests.get(url, params=params)
        data = response.json()
        
        # Ajout des données dans la liste avec l'index correspondant
        weather_data.append({
            "AccidentIndex": index,
            "WeatherData": data
        })
    except Exception as e:
        # Gestion des erreurs pour éviter les arrêts brutaux
        print(f"Erreur pour l'index {index}: {e}")

# Sauvegarde des résultats dans un fichier JSON
with open('weather_data.json', 'w') as json_file:
    json.dump(weather_data, json_file, indent=4)

print("Données météorologiques ajoutées avec succès !")


Données météorologiques ajoutées avec succès !
